# Deep Learning models using Keras

Keras is a user-friendly neural network library written in Python. We will build a regression model to predict an employee's wage per hour.

Note: The datasets we will be using are relatively clean, so we will not perform any data preprocessing in order to get our data ready for modeling. Datasets that you will use in future projects may not be so clean - for example, they may have missing values - so you may need to use data preprocessing techniques to alter your datasets to get more accurate results.

# Regression Model using Keras

For our regression deep learning model, the first step is to read in the data we will use as input. For this example, we are using the 'hourly wages' dataset. To start, we will use Pandas to read in the data. I will not go into detail on Pandas, but it is a library you should become familiar with if you're looking to dive further into data science and machine learning.

'df' stands for dataframe. Pandas read in the CSV file as a dataframe. The 'head()' function will show the first 5 rows of the dataframe so you can check that the data has been read in properly and can take an initial look at how the data is structured.

In [91]:
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

from keras import utils 
from keras.utils import to_categorical

In [92]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/IntroToDNNwKeras.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 429, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 429 (delta 64), reused 20 (delta 20), pack-reused 337
Receiving objects: 100% (429/429), 12.13 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (235/235), done.
/content/cloned-repo/cloned-repo/cloned-repo


In [ ]:
#read in training data
wage_df = pd.read_csv('/content/cloned-repo/hourly_wage')

#view data structure
wage_df.head()
print(wage_df.shape)


(534, 10)


Next, we need to split up our dataset into inputs (train_X) and our target (train_y). Our input will be every column except 'wage_per_hour' because 'wage_per_hour' is what we will be attempting to predict. Therefore, 'wage_per_hour' will be our target.

We will use the pandas 'drop' function to drop the column 'wage_per_hour' from our dataframe and store it in the variable 'train_X'. This will be our input.

In [ ]:
#create a dataframe with all training data except the target column
wage_data = wage_df.drop(columns=['wage_per_hour'])

#check that the target variable has been removed
wage_data.head()

,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
0,0,8,21,35,1,1,0,1,0
1,0,9,42,57,1,1,0,1,0
2,0,12,1,19,0,0,0,1,0
3,0,12,4,22,0,0,0,0,0
4,0,12,17,35,0,1,0,0,0


In [ ]:
#create a label column with only the target column
wage_y = wage_df[['wage_per_hour']]

#view dataframe
wage_y.head()

,wage_per_hour
0,5.10
1,4.95
2,6.67
3,4.00
4,7.50


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    wage_data, wage_y, test_size=0.20)

The model type that we will be using is Sequential. Sequential is the easiest way to build a model in Keras. It allows you to build a model layer by layer. Each layer has weights that correspond to the layer the follows it.

We use the 'add()' function to add layers to our model. We will add two layers and an output layer.

'Dense' is the layer type. Dense is a standard layer type that works for most cases. In a dense layer, all nodes in the previous layer connect to the nodes in the current layer.

We have 10 nodes in each of our input layers. This number can also be in the hundreds or thousands. Increasing the number of nodes in each layer increases model capacity. I will go into further detail about the effects of increasing model capacity shortly.

'Activation' is the activation function for the layer. An activation function allows models to take into account nonlinear relationships. For example, if you are predicting diabetes in patients, going from age 10 to 11 is different than going from age 60â??61.

The activation function we will be using is ReLU or Rectified Linear Activation. Although it is two linear pieces, it has been proven to work well in neural networks.

The first layer needs an input shape. The input shape specifies the number of rows and columns in the input. The number of columns in our input is stored in 'n_cols'. There is nothing after the comma which indicates that there can be any amount of rows.

The last layer is the output layer. It only has one node, which is for our prediction.

Next, we need to compile our model. Compiling the model takes two parameters: optimizer and loss.

The optimizer controls the learning rate. We will be using 'Adam' as our optimizer. Adam is generally a good optimizer to use for many cases. The Adam optimizer adjusts the learning rate throughout training.

The learning rate determines how fast the optimal weights for the model are calculated. A smaller learning rate may lead to more accurate weights (up to a certain point), but the time it takes to compute the weights will be longer.

For our loss function, we will use 'mean_squared_error'. It is calculated by taking the average squared difference between the predicted and actual values. It is a popular loss function for regression problems. The closer to 0 this is, the better the model performed.



In [ ]:
#create model
model = Sequential()

#get number of columns in training data
n_cols = X_train.shape[1]

#add model layers
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

#compile model using mse as a measure of model performance
model.compile(optimizer='adam', loss='mean_squared_error', metrics=["mean_squared_error"])

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

Now we will train our model. To train, we will use the 'fit()' function on our model with the following five parameters: training data (train_X), target data (train_y), validation split, the number of epochs and callbacks.

The validation split will randomly split the data into use for training and testing. During training, we will be able to see the validation loss, which gives the mean squared error of our model on the validation set. We will set the validation split at 0.2, which means that 20% of the training data we provide in the model will be set aside for testing model performance.

The number of epochs is the number of times the model will cycle through the data. The more epochs we run, the more the model will improve, up to a certain point. After that point, the model will stop improving during each epoch. In addition, the more epochs, the longer the model will take to run. To monitor this, we will use 'early stopping'.

Early stopping will stop the model from training before the number of epochs is reached if the model stops improving. We will set our early stopping monitor to 3. This means that after 3 epochs in a row in which the model doesn't improve, training will stop. Sometimes, the validation loss can stop improving then improve in the next epoch, but after 3 epochs in which the validation loss doesn't improve, it usually won't improve again.

In [ ]:
#train model
model.fit(X_train, y_train, validation_split=0.2, epochs=30, 
          callbacks=[early_stopping_monitor])

Epoch 1/30
11/11 [==============================] - 4s 131ms/step - loss: 42.8771 - mean_squared_error: 42.8771 - val_loss: 32.3946 - val_mean_squared_error: 32.3946
Epoch 2/30
11/11 [==============================] - 0s 38ms/step - loss: 32.8691 - mean_squared_error: 32.8691 - val_loss: 28.9986 - val_mean_squared_error: 28.9986
Epoch 3/30
11/11 [==============================] - 0s 37ms/step - loss: 30.7176 - mean_squared_error: 30.7176 - val_loss: 27.4333 - val_mean_squared_error: 27.4333
Epoch 4/30
11/11 [==============================] - 0s 31ms/step - loss: 29.0981 - mean_squared_error: 29.0981 - val_loss: 25.7390 - val_mean_squared_error: 25.7390
Epoch 5/30
11/11 [==============================] - 0s 32ms/step - loss: 27.5729 - mean_squared_error: 27.5729 - val_loss: 24.4491 - val_mean_squared_error: 24.4491
Epoch 6/30
11/11 [==============================] - 0s 31ms/step - loss: 26.3565 - mean_squared_error: 26.3565 - val_loss: 23.3766 - val_mean_squared_error: 23.3766
Epoch 7/3

Making predictions on new data

If you want to use this model to make predictions on new data, we would use the 'predict()' function, passing in our new data.
The output would be 'wage_per_hour' predictions.

In [ ]:
wage_pred = model.predict(X_test)

4/4 [==============================] - 0s 13ms/step


In [ ]:
y_test_wages=y_test[["wage_per_hour"]].values
#y_test_wages

In [ ]:
#print("prediction", "\tground truth")
#for i in range(len(y_test_wages)):
#  print(wage_pred[i],"\t", y_test_wages[i])

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
rms_dnn_1=np.sqrt(mean_squared_error(y_test, wage_pred))
rms_dnn_1

4.370932860814626

As you increase the number of nodes and layers in a model, the model capacity increases. Increasing model capacity can lead to a more accurate model, up to a certain point, at which the model will stop improving. Generally, the more training data you provide, the larger the model should be. We are only using a tiny amount of data, so our model is pretty small. The larger the model, the more computational capacity it requires and it will take longer to train.

Let's create a new model using the same training data as our previous model. This time, we will add a layer and increase the nodes in each layer to 200. We will train the model to see if increasing the model capacity will improve our validation score.

In [ ]:
#training a new model on the same data to show the effect of increasing model capacity
#create model
model_mc = Sequential()

#add model layers
model_mc.add(Dense(200, activation='relu', input_shape=(n_cols,)))
model_mc.add(Dense(200, activation='relu'))
model_mc.add(Dense(200, activation='relu'))
model_mc.add(Dense(1))

#compile model using mse as a measure of model performance
model_mc.compile(optimizer='adam', loss='mean_squared_error', metrics=["mean_squared_error"])

In [ ]:
#train model
model_mc.fit(X_train, y_train, validation_split=0.2, 
             epochs=30, callbacks=[early_stopping_monitor])

Epoch 1/30
11/11 [==============================] - 4s 224ms/step - loss: 58.4188 - mean_squared_error: 58.4188 - val_loss: 25.6210 - val_mean_squared_error: 25.6210
Epoch 2/30
11/11 [==============================] - 0s 33ms/step - loss: 27.3433 - mean_squared_error: 27.3433 - val_loss: 22.7181 - val_mean_squared_error: 22.7181
Epoch 3/30
11/11 [==============================] - 1s 53ms/step - loss: 24.2522 - mean_squared_error: 24.2522 - val_loss: 21.3029 - val_mean_squared_error: 21.3029
Epoch 4/30
11/11 [==============================] - 0s 33ms/step - loss: 22.7364 - mean_squared_error: 22.7364 - val_loss: 20.1274 - val_mean_squared_error: 20.1274
Epoch 5/30
11/11 [==============================] - 0s 30ms/step - loss: 22.6365 - mean_squared_error: 22.6365 - val_loss: 20.5227 - val_mean_squared_error: 20.5227
Epoch 6/30
11/11 [==============================] - 0s 27ms/step - loss: 22.9991 - mean_squared_error: 22.9991 - val_loss: 21.8230 - val_mean_squared_error: 21.8230
Epoch 7/3

In [ ]:
wage_pred_mc = model_mc.predict(X_test)

4/4 [==============================] - 0s 5ms/step


In [ ]:
rms_dnn=np.sqrt(mean_squared_error(y_test, wage_pred_mc))
rms_dnn

4.596993270986249

In [ ]:
#print("prediction", "\tground truth")
#for i in range(len(y_test_wages)):
#  print(wage_pred_mc[i],"\t", y_test_wages[i])


In [ ]:
from sklearn.linear_model import LinearRegression
model_sklearn=LinearRegression().fit(X_train, y_train)
pred_sklearn=model_sklearn.predict(X_test)

In [ ]:
model_sklearn.score(X_test, y_test)

0.3728298851380055

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
rms_sklearn=np.sqrt(mean_squared_error(y_test, pred_sklearn))
rms_sklearn

4.0879711986224505

In [ ]:
print("sklearn model:", rms_sklearn) 
print("dnn model:",rms_dnn)
print( "first dnn model:", rms_dnn_1)

sklearn model: 4.0879711986224505
dnn model: 4.596993270986249
first dnn model: 4.370932860814626




---



---



# Classification Model using Keras

Now let's move on to building our model for classification. Since many steps will be a repeat from the previous model, I will only go over new concepts.


   Given is the attribute name, attribute type, the measurement unit and a
   brief description.  The number of rings is the value to predict: either
   as a continuous value or as a classification problem.

	Name		Data Type	Meas.	Description
	----		---------	-----	-----------
	Sex		nominal			M, F, and I (infant)
	Length		continuous	mm	Longest shell measurement
	Diameter	continuous	mm	perpendicular to length
	Height		continuous	mm	with meat in shell
	Whole weight	continuous	grams	whole abalone
	Shucked weight	continuous	grams	weight of meat
	Viscera weight	continuous	grams	gut weight (after bleeding)
	Shell weight	continuous	grams	after being dried
	Rings		integer			+1.5 gives the age in years


Predicting the age of abalone from physical measurements. The age of abalone is determined by cutting the shell through the cone, staining it, and counting the number of rings through a microscope -- a boring and time-consuming task. <br>
Other measurements, which are easier to obtain, are used to predict the age. <br>
Further information, such as weather patterns and location (hence food availability) may be required to solve the problem.

In [112]:
#read in training data
train_df_2 = pd.read_csv('abalone.data')

#view data structure
train_df_2.columns

Index(['M', '0.455', '0.365', '0.095', '0.514', '0.2245', '0.101', '0.15',
       '15'],
      dtype='object')

**Convert column 'M' to one-hot encoding**

In [113]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
train_df_2['M'] = LE.fit_transform(train_df_2['M'])
train_df_2

,M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,15
0,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
1,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
2,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
3,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
4,1,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.1200,8
...,...,...,...,...,...,...,...,...,...
4171,0,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4172,2,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4173,2,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4174,0,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [114]:
cols=train_df_2.columns
train_df_2 = train_df_2.astype({'0.455':'float', '0.365':'float', '0.095':'float', '0.514':'float', '0.2245':'float', '0.101':'float', '0.15':'float'})

In [115]:
#create a dataframe with all training data except the target column
train_X_2 = train_df_2.drop(columns=['15'])
train_y = train_df_2[["15"]]
#check that the target variable has been removed
train_X_2.head()

,M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15
0,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
1,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
2,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
3,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055
4,1,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120


In [116]:
train_y

,15
0,7
1,9
2,10
3,7
4,8
...,...
4171,11
4172,10
4173,9
4174,10


When separating the target column, we need to call the 'to_categorical()' function so that column will be 'one-hot encoded'. Currently, a patient with no diabetes is represented with a 0 in the diabetes column and a patient with diabetes is represented with a 1. 

With one-hot encoding, the integer will be removed and a binary variable is inputted for each category. In our case, we have two categories: no diabetes and diabetes.

A patient with no diabetes will be represented by [1 0] and a patient with diabetes will be represented by [0 1].

In [118]:
#one-hot encode target column
train_y_2 = to_categorical(train_y,30)

#vcheck that target column has been converted
train_y_2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

The last layer of our model has 2 nodes - one for each option: the patient has diabetes or they don't.

The activation is 'softmax'. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has a higher probability.

We will use 'categorical_crossentropy' for our loss function. This is the most common choice for classification. A lower score indicates that the model is performing better.

To make things even easier to interpret, we will use the 'accuracy' metric to see the accuracy score on the validation set at the end of each epoch.

In [119]:
print(train_X_2.shape)
outputs=train_y_2.shape
print(outputs)

(4176, 8)
(4176, 30)


In [120]:
#create model
model_2 = Sequential()
#get number of columns in training data
inputs=train_X_2.shape[1]
#add layers to model
model_2.add(Dense(250, activation='relu', input_shape=(inputs,)))
model_2.add(Dense(250, activation='relu'))
model_2.add(Dense(250, activation='relu'))
model_2.add(Dense(250, activation='relu'))
model_2.add(Dense(30, activation='softmax'))
model_2.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 250)               2250      
                                                                 
 dense_30 (Dense)            (None, 250)               62750     
                                                                 
 dense_31 (Dense)            (None, 250)               62750     
                                                                 
 dense_32 (Dense)            (None, 250)               62750     
                                                                 
 dense_33 (Dense)            (None, 30)                7530      
                                                                 
Total params: 198,030
Trainable params: 198,030
Non-trainable params: 0
_________________________________________________________________


In [121]:
#compile model using accuracy to measure model performance
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [122]:
import numpy as np
y = np.asarray(train_y_2).astype(np.int32)
print(y[1])

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
#train_X_2 = np.array([np.array(val) for val in train_X_2])

In [123]:
#train model
model_2.fit(train_X_2, train_y_2, epochs=120)

Epoch 1/120
131/131 [==============================] - 2s 7ms/step - loss: 2.4245 - accuracy: 0.1954
Epoch 2/120
131/131 [==============================] - 2s 12ms/step - loss: 2.1052 - accuracy: 0.2356
Epoch 3/120
131/131 [==============================] - 2s 18ms/step - loss: 2.0229 - accuracy: 0.2586
Epoch 4/120
114/131 [=========================>....] - ETA: 0s - loss: 1.9827 - accuracy: 0.2492

KeyboardInterrupt: ignored

**Assignment**<br>
Use the SkLearn library model for classification to compare with the accuracy of the DNN model. 

In [ ]:
from sklearn.linear_model import LogisticRegression
model_sklearn=LogisticRegression().fit(train_X_2, train_y)
#pred_sklearn=model_sklearn.predict(X_test)
